In [1]:
using Plots, ComplexPhasePortrait, ApproxFun, SingularIntegralEquations, DifferentialEquations
gr();

# M3M6: Methods of Mathematical Physics

$$
\def\dashint{{\int\!\!\!\!\!\!-\,}}
\def\infdashint{\dashint_{\!\!\!-\infty}^{\,\infty}}
\def\D{\,{\rm d}}
\def\E{{\rm e}}
\def\dx{\D x}
\def\dt{\D t}
\def\dz{\D z}
\def\C{{\mathbb C}}
\def\R{{\mathbb R}}
\def\CC{{\cal C}}
\def\HH{{\cal H}}
\def\I{{\rm i}}
\def\qqqquad{\qquad\qquad}
\def\qqfor{\qquad\hbox{for}\qquad}
\def\qqwhere{\qquad\hbox{where}\qquad}
\def\Res_#1{\underset{#1}{\rm Res}}\,
\def\sech{{\rm sech}\,}
\def\acos{\,{\rm acos}\,}
\def\vc#1{{\mathbf #1}}
\def\ip<#1,#2>{\left\langle#1,#2\right\rangle}
\def\norm#1{\left\|#1\right\|}
\def\half{{1 \over 2}}
$$

Dr. Sheehan Olver
<br>
s.olver@imperial.ac.uk

Office Hours: 3-4pm Mondays, 11-12am Thursdays, Huxley 6M40
<br>
Website: https://github.com/dlfivefifty/M3M6LectureNotes


# Lecture 17: Solving differential equations with orthogonal polynomials


This lecture we do the following:

1. Recurrence relationships for Chebyshev and ultrashperical polynomials (continued)
    - Conversion
    - Three-term recurrence and Jacobi operators
2. Application: solving differential equations
    - First order constant coefficients differential equations
    - Second order constant coefficient differential equations with boundary conditions
    - Non-constant coefficients
    


That is, we introduce recurrences related to ultraspherical polynomials. This allows us to represent general linear differential equations as almost-banded systems.

## Recurrence relationships for Chebyshev and ultraspherical polynomials

Recall from last lecture we had the following formulae for derivatives:

**Proposition (Chebyshev derivative)** $$T_n'(x) = n U_{n-1}(x)$$

**Proposition (Ultraspherical derivative)** 
$${\D \over \dx} C_n^{(\lambda)}(x) = 2 \lambda  C_{n-1}^{(\lambda+1)}(x)$$

Like the three-term recurrence and Jacobi operators, it is useful to express this in matrix form. That is, for the derivatives of $T_n(x)$ we get
$$
{\D \over \dx}  \begin{pmatrix} T_0(x) \\ T_1(x) \\ T_2(x) \\ \vdots \end{pmatrix}= \begin{pmatrix}
0 \cr
1 \cr 
& 2 \cr
&& 3 \cr
&&&\ddots 
\end{pmatrix} \begin{pmatrix} U_0(x) \\ U_1(x) \\ U_2(x) \\ \vdots \end{pmatrix} 
$$
which let's us know that, for 
$$
f(x) = (T_0(x),T_1(x),\ldots) \begin{pmatrix} f_0\\f_1\\\vdots \end{pmatrix}
$$
we have a derivative operator in coefficient space as
$$
f'(x) = (U_0(x),U_1(x),\ldots)\begin{pmatrix}
0 & 1 \cr 
&& 2 \cr
&&& 3 \cr
&&&&\ddots 
\end{pmatrix}  \begin{pmatrix} f_0\\f_1\\\vdots \end{pmatrix}
$$

Note that in ApproxFun.jl we can construct these operators rather nicely:

In [2]:
D = Derivative()
(D*f)(0.1)

LoadError: [91mUndefVarError: f not defined[39m

In [3]:
D : Chebyshev() → Ultraspherical(1) # note it should print a 0 in the (1,1) entry

ConcreteDerivative:Chebyshev(【-1.0,1.0】)→Ultraspherical(1,【-1.0,1.0】)
   1.0                                           
        2.0                                      
             3.0                                 
                  4.0                            
                       5.0                       
                            6.0                  
                                 7.0             
                                      8.0        
                                           9.0   
                                                ⋱
                                                ⋱

### Conversion



We can convert between any two polynomial bases using a lower triangular operator, because their span's are equivalent. In the case of Chebyshev and ultraspherical polynomials, they have the added property that they are banded.

**Proposition (Chebyshev T-to-U conversion)** 
\begin{align*}
 T_0(x) &= U_0(x) \\
 T_1(x) &= {U_1(x) \over 2} \\
 T_n(x) &= {U_n(x) \over 2} - {U_{n-2}(x) \over 2}
\end{align*}

**Proof** 

Before we do the proof, note that the fact that there are limited non-zero entries follows immediately: if $m < n-2$ we have
$$
\ip<T_n, U_m>_{\rm U} = \ip<T_n, (1-x^2)U_m>_{\rm T} = 0
$$

To actually determine the entries, we use the trigonometric formulae. Recall for $x = (z + z^{-1})/2$, $z = \E^{\I \theta}$, we have
\begin{align*}
T_n(x) &= \cos n \theta = {z^{-n} + z^n \over 2}\\
U_n(x) &= {\sin (n+1) \theta \over \sin \theta} = {z^{n+1} - z^{-n-1} \over z - z^{-1}} = z^{-n} + z^{2-n} + \cdots +  \cdots + z^{n-2} + z^n 
\end{align*}
The result follows immediately.

⬛️

**Corollary (Ultrapherical λ-to-(λ+1) conversion)**
$$
C_n^{(\lambda)}(x) = {\lambda \over n+ \lambda} (C_n^{(\lambda+1)}(x) - C_{n-2}^{(\lambda+1)}(x))
$$

**Proof** This follows from differentiating the previous result. For example:
\begin{align*}
 {\D\over \dx} T_0(x) &= {\D\over \dx} U_0(x) \\
 {\D\over \dx} T_1(x) &= {\D\over \dx} {U_1(x) \over 2} \\
{\D\over \dx} T_n(x) &= {\D\over \dx} \left({U_n(x) \over 2} - {U_{n-2} \over 2} \right)
\end{align*}
becomes
\begin{align*}
    0 &= 0\\
    U_0(x) &= C_1^{(2)}(x) \\
   n U_{n-1}(x) &= C_{n-1}^{(2)}(x)  - C_{n-3}^{(2)}(x)
\end{align*}

Differentiating this repeatedly completes the proof.

⬛️


Note we can write this in matrix form, for example, we have
$$
\underbrace{\begin{pmatrix}1 \cr
                    0 & \half\cr
                       -\half & 0 & \half \cr
                           &\ddots &\ddots & \ddots\end{pmatrix} }_{S_0^\top} \begin{pmatrix} 
                           U_0(x) \\ U_1(x) \\ U_2(x) \\ \vdots \end{pmatrix}  =  \begin{pmatrix} T_0(x) \\ T_1(x) \\ T_2(x) \\ \vdots \end{pmatrix}
$$

therefore,
$$
f(x) =  (T_0(x),T_1(x),\ldots) \begin{pmatrix} f_0\\f_1\\\vdots \end{pmatrix} =  (U_0(x),U_1(x),\ldots) S_0 \begin{pmatrix} f_0\\f_1\\\vdots \end{pmatrix}
$$

Again, we can construct this nicely in ApproxFun:

In [4]:
S₀ = I : Chebyshev() → Ultraspherical(1)

ConcreteConversion:Chebyshev(【-1.0,1.0】)→Ultraspherical(1,【-1.0,1.0】)
 1.0  0.0  -0.5                                             
      0.5   0.0  -0.5                                       
            0.5   0.0  -0.5                                 
                  0.5   0.0  -0.5                           
                        0.5   0.0  -0.5                     
                              0.5   0.0  -0.5               
                                    0.5   0.0  -0.5         
                                          0.5   0.0  -0.5   
                                                0.5   0.0  ⋱
                                                      0.5  ⋱
                                                           ⋱

In [5]:
f = Fun(exp, Chebyshev())
g = S₀*f

Fun(Ultraspherical(1,【-1.0,1.0】),[1.13032, 0.542991, 0.133011, 0.021897, 0.00271463, 0.000269864, 2.23891e-5, 1.5937e-6, 9.93309e-8, 5.5059e-9, 2.74775e-10, 1.24699e-11, 5.19555e-13, 1.99597e-14])

In [6]:
g(0.1) - exp(0.1)

0.0

### Ultraspherical Three-term recurrence 

**Theorem (three-term recurrence for Chebyshev U)** 
\begin{align*}
x U_0(x) &= {U_1(x) \over 2} \\
x U_n(x) &= {U_{n-1}(x) \over 2} + {U_{n+1}(x) \over 2}
\end{align*}

**Proof**
Differentiating
\begin{align*}
 x T_0(x) &= T_1(x) \\
x T_n(x)  &=  {T_{n-1}(x) \over 2} + {T_{n+1}(x) \over 2}
\end{align*}
we get
\begin{align*}
  T_0(x) &= U_0(x) \\
 T_n(x) + n x U_{n-1}(x)  &=  {(n-1) U_{n-2}(x) \over 2} + {(n+1) U_n(x) \over 2}
\end{align*}
substituting in the conversion $T_n(x) = (U_n(x) - U_{n-2}(x))/2$ we get
\begin{align*}
  T_0(x) &= U_0(x) \\
 n x U_{n-1}(x)  &=  {(n-1) U_{n-2}(x) \over 2} + {(n+1) U_n(x) \over 2} - (U_n(x) - U_{n-2}(x))/2 = {n U_{n-2}(x) \over 2} + {n U_n(x) \over 2}
\end{align*}

⬛️

Differentiating this theorem again and applying the conversion we get the following

**Corollary (three-term recurrence for ultrashperical)** 
\begin{align*}
x C_0^{(\lambda)}(x) &= {1 \over 2\lambda } C_1^{(\lambda)}(x) \\
 x C_n^{(\lambda)}(x) &=  {n+2\lambda-1 \over 2(n+\lambda)} C_{n-1}^{(\lambda)}(x) + {n+1 \over 2(n+\lambda)} C_{n+1}^{(\lambda)}(x) 
\end{align*}


Here's an example of the Jacobi operator (which is the transpose of the multiplciation by $x$ operator):

In [7]:
Multiplication(Fun(), Ultraspherical(2))'

TransposeOperator:Ultraspherical(2,【-1.0,1.0】)→Ultraspherical(2,【-1.0,1.0】)
 0.0       0.25                       …                                     
 0.666667  0.0    0.333333                                                  
           0.625  0.0       0.375                                           
                  0.6       0.0                                             
                            0.583333                                        
                                      …  0.428571                           
                                         0.0       0.4375                   
                                         0.555556  0.0     0.444444         
                                                   0.55    0.0       0.45   
                                                           0.545455  0.0   ⋱
                                      …                               ⋱    ⋱

## Application: solving differential equations

The preceding results allowed us to represent 

1. Differentiation
2. Conversion
3. Multiplication

as banded operators. We will see that we can combine these, along with 

4\. Evaluation

to solve ordinary differential equations.

### First order, constant coefficient differential equations

Consider the simplest ODE:
\begin{align*}
u(0) &= 0 \\
u'(x) - u(x) &= 0 
\end{align*}
and suppose  represent $u(x)$ in its Chebyshev expansion, with to be determined coefficents. In other words, we want to calculate coefficients $u_k$ such that
$$
u(x) = \sum_{k=0}^\infty u_k T_k(x) = (T_0(x), T_1(x), \ldots) \begin{pmatrix} u_0 \\ u_1 \\ \vdots \end{pmatrix}
$$
In this case we know that $u(x) = \E^x$, but we would still need other means to calculate $u_k$ (They are definitely not as simple as Taylor series coefficients).

We can express the constraints as acting on the coefficients. For example, we have
$$
u(0) = (T_0(0), T_1(0), \ldots) \begin{pmatrix} u_0\\u_1\\\vdots \end{pmatrix} = (1,0,-1,0,1,\ldots)  \begin{pmatrix} u_0\\u_1\\\vdots \end{pmatrix} 
$$
We also have 
$$u'(x) = (U_0(x),U_1(x),\ldots) \begin{pmatrix}
0 & 1 \cr 
&& 2 \cr
&&& 3 \cr
&&&&\ddots 
\end{pmatrix}\begin{pmatrix} u_0\\u_1\\\vdots \end{pmatrix} 
$$
To represent $u'(x) - u(x)$, we need to make sure the bases are compatible. In other words, we want to express $u(x)$ in it's $U_k(x)$ basis using the conversion operator $S_0$:
$$u(x) = (U_0(x),U_1(x),\ldots) \begin{pmatrix}
    1 &0 & -\half \cr 
& \half & 0 & -\half \cr
&&\ddots & \ddots & \ddots
\end{pmatrix}\begin{pmatrix} u_0\\u_1\\\vdots \end{pmatrix} 
$$

Which gives us, 
$$
u'(x) - u(x) =  (U_0(x),U_1(x),\ldots)  \begin{pmatrix}
    -1 &1 & \half \cr 
& -\half & 2 & \half \cr
&& -\half & 3 & \half \cr
&&&\ddots & \ddots & \ddots
\end{pmatrix} \begin{pmatrix} u_0\\u_1\\\vdots \end{pmatrix} 
$$


Combing the differential part and the evaluation part, we arrive at an (infinite) system of equations for the coefficients $u_0,u_1,\dots$:
$$
\begin{pmatrix}
      1 & 0 & -1 & 0 & 1 & \cdots \\
    -1 &1 & \half \cr 
& -\half & 2 & \half \cr
&& -\half & 3 & \half \cr
&&&\ddots & \ddots & \ddots
\end{pmatrix} \begin{pmatrix} u_0\\u_1\\\vdots \end{pmatrix}  = \begin{pmatrix} 1 \\ 0 \\ 0 \\ \vdots \end{pmatrix}
$$

How to solve this system is outside the scope of this course (though a simple approach is to truncate the infinite system to finite systems). We can however do this in ApproxFun:

In [8]:
B  = Evaluation(0.0) : Chebyshev()
D  = Derivative() : Chebyshev() → Ultraspherical(1)
S₀ = I : Chebyshev() → Ultraspherical(1)
L = [B; 
     D - S₀]

InterlaceOperator:Chebyshev(【-1.0,1.0】)→2-element ArraySpace:
 ConstantSpace(Point(0.0))   
 Ultraspherical(1,【-1.0,1.0】)
  1.0   0.0  -1.0   0.0   1.0   0.0  -1.0   0.0   1.0  0.0  ⋯
 -1.0   1.0   0.5   0.0   0.0   0.0   0.0   0.0   0.0  0.0  ⋱
  0.0  -0.5   2.0   0.5   0.0   0.0   0.0   0.0   0.0  0.0  ⋱
  0.0   0.0  -0.5   3.0   0.5   0.0   0.0   0.0   0.0  0.0  ⋱
  0.0   0.0   0.0  -0.5   4.0   0.5   0.0   0.0   0.0  0.0  ⋱
  0.0   0.0   0.0   0.0  -0.5   5.0   0.5   0.0   0.0  0.0  ⋱
  0.0   0.0   0.0   0.0   0.0  -0.5   6.0   0.5   0.0  0.0  ⋱
  0.0   0.0   0.0   0.0   0.0   0.0  -0.5   7.0   0.5  0.0  ⋱
  0.0   0.0   0.0   0.0   0.0   0.0   0.0  -0.5   8.0  0.5  ⋱
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -0.5  9.0  ⋱
   ⋮     ⋱     ⋱     ⋱     ⋱     ⋱     ⋱     ⋱     ⋱    ⋱   ⋱

In [9]:
cos(-0.1), -sin(0.2-0.1)

(0.9950041652780258, -0.09983341664682815)

In [10]:
u = L \ [1; 0]
L*u

Fun(2-element ArraySpace:
 ConstantSpace(Point(0.0))   
 Ultraspherical(1,【-1.0,1.0】),[1.0, 2.22045e-16, -6.93889e-18, -7.28584e-17, 9.75782e-18, -4.13352e-19, -1.37643e-20, 7.04096e-21, -3.70577e-22, -3.20532e-24, 1.8967e-24, 4.24092e-26, 4.61957e-27, -1.96377e-27, -4.73752e-26, -1.32819e-24, -3.98931e-23, -1.27791e-21, -4.34887e-20, 6.03922e-22])

In [11]:
u(0.1) - exp(0.1)

2.220446049250313e-16

Note we can incorporate right-hand sides as well, for example, to solve $u'(x) - u(x) = f(x)$, by expanding $f$ in its Chebyshev U series.

### Second-order constanst coefficient equations

This approach extends to second-order constant-coefficient equations by using ultraspherical polynomials.  Consider
\begin{align*}
u(-1) &= 1\\
u(1) &= 0\\
u''(x) + u'(x)  + u(x) &= 0
\end{align*}
Evaluation works as in the first-order case. To handle second-derivatives, we need $C^{(2)}$ polynomials:

In [12]:
D₀ = Derivative() : Chebyshev() → Ultraspherical(1)
D₁ = Derivative() : Ultraspherical(1) → Ultraspherical(2)
D₁*D₀  # 2 zeros not printed in (1,1) and (1,2) entry

ConcreteDerivative:Chebyshev(【-1.0,1.0】)→Ultraspherical(2,【-1.0,1.0】)
     4.0                                           
          6.0                                      
               8.0                                 
                    10.0                           
                          12.0                     
                                14.0               
                                      16.0         
                                            18.0   
                                                  ⋱
                                                  ⋱
                                                  ⋱

For the identity operator, we use two conversion operators:

In [13]:
S₀ = I : Chebyshev() → Ultraspherical(1)
S₁ = I : Ultraspherical(1) → Ultraspherical(2)
S₁*S₀

ConversionWrapper:Chebyshev(【-1.0,1.0】)→Ultraspherical(2,【-1.0,1.0】)
 1.0  0.0   -0.666667   0.0    …                                    
      0.25   0.0       -0.375                                       
             0.166667   0.0                                         
                        0.125      0.0833333                        
                                   0.0         0.0714286            
                               …  -0.145833    0.0         0.0625   
                                   0.0        -0.126984    0.0     ⋱
                                   0.0625      0.0        -0.1125  ⋱
                                               0.0555556   0.0     ⋱
                                                           0.05    ⋱
                               …                                   ⋱

And for the first derivative, we use a derivative and then a conversion:

In [14]:
S₁*D₀  # or could have been D₁*S₀

TimesOperator:Chebyshev(【-1.0,1.0】)→Ultraspherical(2,【-1.0,1.0】)
   1.0  0.0  -1.0                                       
        1.0   0.0  -1.0                                 
              1.0   0.0  -1.0                           
                    1.0   0.0  -1.0                     
                          1.0   0.0  -1.0               
                                1.0   0.0  -1.0         
                                      1.0   0.0  -1.0   
                                            1.0   0.0  ⋱
                                                  1.0  ⋱
                                                       ⋱
                                                       ⋱

Putting everything together we get:

In [15]:
B₋₁ = Evaluation(-1) : Chebyshev()
B₁ = Evaluation(1) : Chebyshev()
# u(-1) 
# u(1)
# u'' + u' +u

L = [B₋₁; 
     B₁; 
     D₁*D₀ + S₁*D₀ + S₁*S₀]

InterlaceOperator:Chebyshev(【-1.0,1.0】)→3-element ArraySpace:
 ConstantSpace(Point(-1))    
 ConstantSpace(Point(1))     
 Ultraspherical(2,【-1.0,1.0】)
 1.0  -1.0   1.0       -1.0    …  -1.0         1.0        -1.0     ⋯
 1.0   1.0   1.0        1.0        1.0         1.0         1.0     ⋱
 1.0   1.0   3.33333   -1.0        0.0         0.0         0.0     ⋱
 0.0   0.25  1.0        5.625      0.0         0.0         0.0     ⋱
 0.0   0.0   0.166667   1.0        0.0         0.0         0.0     ⋱
 0.0   0.0   0.0        0.125  …   0.0833333   0.0         0.0     ⋱
 0.0   0.0   0.0        0.0       -1.0         0.0714286   0.0     ⋱
 0.0   0.0   0.0        0.0       13.8542     -1.0         0.0625  ⋱
 0.0   0.0   0.0        0.0        1.0        15.873      -1.0     ⋱
 0.0   0.0   0.0        0.0        0.0625      1.0        17.8875  ⋱
  ⋮     ⋱     ⋱          ⋱     …    ⋱           ⋱           ⋱      ⋱

In [16]:
u = L \ [1.0,0.0,0.0]
plot(u)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

### Variable coefficients

Consider the Airy ODE 
\begin{align*}
u(-1) &= 1\\
u(1) &= 0\\
u''(x) - xu(x) &= 0
\end{align*}

to handle, this, we need only use the Jacobi operator to represent multiplication by $x$:

In [17]:
x = Fun()
Jᵗ = Multiplication(x) : Chebyshev() → Chebyshev()  # transpose of the Jacobi operator

ConcreteMultiplication:Chebyshev(【-1.0,1.0】)→Chebyshev(【-1.0,1.0】)
 0.0  0.5                                           
 1.0  0.0  0.5                                      
      0.5  0.0  0.5                                 
           0.5  0.0  0.5                            
                0.5  0.0  0.5                       
                     0.5  0.0  0.5                  
                          0.5  0.0  0.5             
                               0.5  0.0  0.5        
                                    0.5  0.0  0.5   
                                         0.5  0.0  ⋱
                                               ⋱   ⋱

In [18]:
L = [B₋₁;   # u(-1)
     B₁ ;   # u(1)
     D₁*D₀ - S₁*S₀*Jᵗ]   # u'' - x*u

InterlaceOperator:Chebyshev(【-1.0,1.0】)→3-element ArraySpace:
 ConstantSpace(Point(-1))    
 ConstantSpace(Point(1))     
 Ultraspherical(2,【-1.0,1.0】)
  1.0   -1.0         1.0     -1.0    1.0        …   1.0        -1.0        ⋯
  1.0    1.0         1.0      1.0    1.0            1.0         1.0        ⋱
  0.0   -0.166667    4.0      0.25   0.0            0.0         0.0        ⋱
 -0.25   0.0         0.0625   6.0    0.125          0.0         0.0        ⋱
  0.0   -0.0833333   0.0      0.05   8.0            0.0         0.0        ⋱
  0.0    0.0        -0.0625   0.0    0.0416667  …  -0.0416667   0.0        ⋱
  0.0    0.0         0.0     -0.05   0.0            0.0        -0.0357143  ⋱
  0.0    0.0         0.0      0.0   -0.0416667      0.0416667   0.0        ⋱
  0.0    0.0         0.0      0.0    0.0           16.0         0.0357143  ⋱
  0.0    0.0         0.0      0.0    0.0            0.025      18.0        ⋱
   ⋮      ⋱           ⋱        ⋱      ⋱         …    ⋱           ⋱         ⋱

In [19]:
u = L \ [1.0;0.0;0.0]
plot(u; legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8

If we introduce a small parameter, that is, solve
\begin{align*}
u(-1) &= 1\\
u(1) &= 0\\
\epsilon u''(x) - xu(x) &= 0
\end{align*}
we can see pretty hard to compute solutions:

In [20]:
ε = 1E-6
L = [B₋₁; 
     B₁ ; 
     ε*D₁*D₀ - S₁*S₀*Jᵗ]

u = L \ [1.0;0.0;0.0]
plot(u; legend=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#cli

Because of the banded structure, this can be solved fast:

In [21]:
ε = 1E-10
L = [B₋₁; 
     B₁ ; 
     ε*D₁*D₀ - S₁*S₀*Jᵗ]


@time u = L \ [1.0;0.0;0.0]
@show ncoefficients(u);

  0.079070 seconds (5.23 k allocations: 80.307 MiB, 16.11% gc time)
ncoefficients(u) = 62496


To handle other variable coefficients, first consider a polynomial $p(x)$. If Multiplication by $x$ is represented by multiplying the coefficients by $J^\top$, then multiplication by $p$ is represented by $p(J^\top)$:

In [22]:
M = -I + Jᵗ + (Jᵗ)^2  # represents -1+x+x^2

PlusOperator:Chebyshev(【-1.0,1.0】)→Chebyshev(【-1.0,1.0】)
 -0.5   0.5    0.25                                                    
  1.0  -0.25   0.5    0.25                                             
  0.5   0.5   -0.5    0.5    0.25                                      
        0.25   0.5   -0.5    0.5    0.25                               
               0.25   0.5   -0.5    0.5    0.25                        
                      0.25   0.5   -0.5    0.5    0.25                 
                             0.25   0.5   -0.5    0.5    0.25          
                                    0.25   0.5   -0.5    0.5    0.25   
                                           0.25   0.5   -0.5    0.5   ⋱
                                                  0.25   0.5   -0.5   ⋱
                                                          ⋱      ⋱    ⋱

In [23]:
ε = 1E-6
L = [B₋₁; 
     B₁ ; 
     ε*D₁*D₀ - S₁*S₀*M]

@time u = L \ [1.0;0.0;0.0]

@show ε*u''(0.1) - (-1+0.1+0.1^2)*u(0.1)
plot(u)

  0.014591 seconds (7.67 k allocations: 2.243 MiB)
ε * ((u')')(0.1) - (-1 + 0.1 + 0.1 ^ 2) * u(0.1) = -1.3974932322469158e-14


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip8502)" style="stroke:#009af9; stroke-width:1; stroke-op

For other smooth functions, we first approximate in a polynomial basis,  and without loss of generality we use Chebyshev T basis. For example, consider 
\begin{align*}
u(-1) &= 1\\
u(1) &= 0\\
\epsilon u''(x) - \E^x u(x) &= 0
\end{align*}
where
$$
\E^x  \approx p(x) = \sum_{k=0}^{m-1} p_k T_k(x)
$$
Evaluating at a point $x$, recall Clenshaw's algorithm:
\begin{align*}
\gamma_{n-1} &= 2p_{n-1} \\
\gamma_{n-2} &= 2p_{n-2} + 2x \gamma_{n-1} \\
\gamma_{n-3} &= 2 p_{n-3} + 2x \gamma_{n-2} - \gamma_{n-1} \\
& \vdots \\
\gamma_1 &= p_1 + x \gamma_2 - \half \gamma_3 \\
p(x) = \gamma_0 &= p_0 + x \gamma_1 - \half \gamma_2
\end{align*}
If multiplication by $x$ becomes $J^\top$, then multiplication by $p(x)$ becomes $p(J^\top)$, and hence we calculate:\
\begin{align*}
\Gamma_{n-1} &= 2p_{n-1}I \\
\Gamma_{n-2} &= 2p_{n-2}I + 2J^\top \Gamma_{n-1} \\
\Gamma_{n-3} &= 2 p_{n-3}I + 2J^\top \Gamma_{n-2} - \Gamma_{n-1} \\
& \vdots \\
\Gamma_1 &= p_1I + J^\top \Gamma_2 - \half \Gamma_3 \\
p(J^\top) = \Gamma_0 &= p_0 + x \Gamma_1 - \half \Gamma_2
\end{align*}

Here is an example:

In [24]:
p = Fun(exp, Chebyshev()) # polynomial approximation to exp(x)
M = Multiplication(p) : Chebyshev() # constructed using Clenshaw:

ConcreteMultiplication:Chebyshev(【-1.0,1.0】)→Chebyshev(【-1.0,1.0】)
 1.26607      0.565159     0.135748     …  9.96062e-8   5.51839e-9   ⋱
 1.13032      1.40181      0.587328        1.60474e-6   9.98815e-8   ⋱
 0.271495     0.587328     1.2688          2.24889e-5   1.59923e-6   ⋱
 0.0443368    0.138485     0.565431        0.000271463  2.24887e-5   ⋱
 0.00547424   0.0224399    0.13577         0.00273712   0.000271463  ⋱
 0.000542926  0.00275961   0.02217      …  0.0221684    0.00273712   ⋱
 4.49773e-5   0.000273062  0.00273722      0.135748     0.0221684    ⋱
 3.19844e-6   2.25883e-5   0.000271469     0.565159     0.135748     ⋱
 1.99212e-7   1.60474e-6   2.24889e-5      1.26607      0.565159     ⋱
 1.10368e-8   9.98815e-8   1.59923e-6      0.565159     1.26607      ⋱
  ⋱            ⋱            ⋱           …   ⋱            ⋱           ⋱

In [25]:
ApproxFun.bandwidths(M) # still banded

(13, 13)

In [26]:
ε = 1E-6
L = [B₋₁; 
     B₁ ; 
     ε*D₁*D₀ + S₁*S₀*M]

@time u = L \ [1.0;0.0;0.0]

@show ε*u''(0.1) + exp(0.1)*u(0.1)
plot(u)

  0.006636 seconds (16.79 k allocations: 5.305 MiB)
ε * ((u')')(0.1) + exp(0.1) * u(0.1) = 7.105427357601002e-15


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip8702)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" po